In [1]:
import numpy as np
import pandas as pd
import torch
import mne
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGRegressor
from braindecode.preprocessing import preprocess, Preprocessor, exponential_moving_standardize
from braindecode.datasets import BaseDataset, BaseConcatDataset, create_from_X_y
from braindecode.training.losses import CroppedLoss
from braindecode.models import Deep4Net
from braindecode.models import ShallowFBCSPNet
from braindecode.models.util import to_dense_prediction_model, get_output_shape
from braindecode.util import set_random_seeds, create_mne_dummy_raw

In [2]:
def load_dataset(folder):
    files = os.listdir(folder)
    dataset = np.load(os.path.join(folder, files[0]), allow_pickle=True)
    files.pop(0)
    for file_path in files:
        ds = np.load(os.path.join(folder, file_path), allow_pickle=True)
        dataset[0] = np.concatenate((dataset[0], ds[0]), axis=0)
        dataset[1] = np.concatenate((dataset[1], ds[1]), axis=0)
    inputs = dataset[0]
    targets = dataset[1]
    return inputs, targets

In [3]:
ch_names = ['CP3', 'C3', 'F5', 'PO3', 'PO4', 'F6', 'C4', 'CP4']
inputs, targets = load_dataset('data/train')
inputs_val, targets_val = load_dataset('data/test')

In [4]:
targets.shape

(36,)

In [5]:
emg_dataset = create_from_X_y(
    inputs, targets, drop_last_window=False, sfreq=256, ch_names=ch_names,
    window_stride_samples=392,
    window_size_samples=1000,
)

emg_dataset_val = create_from_X_y(
    inputs, targets, drop_last_window=False, sfreq=256, ch_names=ch_names,
    window_stride_samples=392,
    window_size_samples=1000,
)

factor_new = 1e-3
init_block_size = 1000
low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 50.  # high cut frequency for filtering
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    # Preprocessor('notch_filter', freqs=60),
    Preprocessor('filter_data', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    # Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
    #              factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
# preprocess(emg_dataset[0], preprocessors)
# preprocess(emg_dataset_val, preprocessors)

emg_dataset.description

Creating RawArray with float64 data, n_channels=8, n_times=1447
    Range : 0 ... 1446 =      0.000 ...     5.648 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1248
    Range : 0 ... 1247 =      0.000 ...     4.871 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1296
    Range : 0 ... 1295 =      0.000 ...     5.059 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs

,target
0,1
1,0
2,2
3,1
4,0
5,2
6,1
7,0
8,2
9,1


In [6]:
model_name = "shallow"  # 'shallow' or 'deep'
n_epochs = 30
seed = 20200220

input_window_samples = 1000
batch_size = 3
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

n_chans = 8
# set to how many targets you want to regress (age -> 1, [x, y, z] -> 3)
n_classes = 3

set_random_seeds(seed=seed, cuda=cuda)

# initialize a model, transform to dense and move to gpu
if model_name == "shallow":
    model = ShallowFBCSPNet(
        in_chans=n_chans,
        n_classes=n_classes,
        input_window_samples=input_window_samples,
        n_filters_time=40,
        n_filters_spat=40,
        final_conv_length=35,
    )
    optimizer_lr = 0.000625
    optimizer_weight_decay = 0
elif model_name == "deep":
    model = Deep4Net(
        in_chans=n_chans,
        n_classes=n_classes,
        input_window_samples=input_window_samples,
        n_filters_time=25,
        n_filters_spat=25,
        stride_before_pool=True,
        n_filters_2=int(n_chans * 2),
        n_filters_3=int(n_chans * (2 ** 2.0)),
        n_filters_4=int(n_chans * (2 ** 3.0)),
        final_conv_length=1,
    )
    optimizer_lr = 0.01
    optimizer_weight_decay = 0.0005
else:
    raise ValueError(f'{model_name} unknown')

new_model = torch.nn.Sequential()
for name, module_ in model.named_children():
    if "softmax" in name:
        continue
    new_model.add_module(name, module_)
model = new_model

if cuda:
    model.cuda()

to_dense_prediction_model(model)
n_preds_per_input = get_output_shape(model, n_chans, input_window_samples)[2]
print(n_preds_per_input)

regressor = EEGRegressor(
    model,
    cropped=True,
    criterion=CroppedLoss,
    criterion__loss_function=torch.nn.functional.mse_loss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(emg_dataset_val),
    optimizer__lr=optimizer_lr,
    optimizer__weight_decay=optimizer_weight_decay,
    iterator_train__shuffle=True,
    batch_size=batch_size,
    callbacks=[
        "neg_root_mean_squared_error",
        # seems n_epochs -1 leads to desired behavior of lr=0 after end of training?
        ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)

regressor.fit(emg_dataset, y=None, epochs=n_epochs)

/home/stelath/miniconda3/envs/neuronav/lib/python3.8/site-packages/braindecode/util.py:51: UserWarning: torch.backends.cudnn.benchmark was set to True which may results in lack of reproducibility. In some cases to ensure reproducibility you may need to set torch.backends.cudnn.benchmark to False.
  warn(


392
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...


/home/stelath/miniconda3/envs/neuronav/lib/python3.8/site-packages/braindecode/training/losses.py:32: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return self.loss_function(avg_preds, targets)


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1